# Stereo SLAM

This is a major part of this project and will likely take some time. 

For stereo, feel free to look up existing tutorials that implement this and write your own code here. Do not spend too long tweaking parameters here, focus on getting decent results and move on. You can also use OpenCV functions to backproject to 3D. 

## PART 1: Stereo dense reconstruction

3-D point clouds are very useful in robotics for several tasks such as object detection, motion estimation (3D-3D matching or 3D-2D matching), SLAM, and other forms of scene understanding.  Stereo camerasprovide  us  with  a  convenient  way  to  generate  dense  point  clouds.Densehere,  in  contrast  tosparse,means all the image points are used for the reconstruction.  In this part of the assignment you will begenerating a dense 3D point cloud reconstruction of a scene from stereo images.

#### Procedure: 

<ol>
    <li> Generate a disparity map for each stereo pair.  Use OpenCV (e.g.  StereoSGBM) for this.  Notethat the images provided are already rectified and undistorted. </li>
    <li> Then, using the camera parameters and baseline information generate colored point clouds from each disparity map.  Some points will have invalid disparity values, so ignore them.  Use [Open3D]for storing your point clouds. </li>
    <li> Register (or transform) all the generated point clouds into your world frame by using the providedground truth poses. </li>
    <li> Visualize the registered point cloud data, in color.  Use Open3D for this </li>
</ol>
    
    
Write briefly about how the disparity map is generated (if you used SGBM, write about SGBM).

In [1]:
import os
import math
import glob

import cv2
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

np.set_printoptions(precision=2)

In [2]:
data_root = "../data"

left_image_path = "../data/img2/*.png"
right_image_path = "../data/img3/*.png"

pose_file = "../data/poses.txt"

In [3]:
def get_images(path):
    filenames = []
    for filename in glob.glob(path):
        filenames.append(filename)
    filenames.sort()

    images = []
    for filename in filenames:
        image = cv2.imread(filename)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        images.append(image)
    return images

def get_poses(pose_file):
    f = open(pose_file, "r")
    lines = f.readlines()
    pose_info = []
    for line in lines:
        temp = line.split()
        pose = []
        pose.append(temp[0:4])
        pose.append(temp[4:8])
        pose.append(temp[8:12])
        pose_info.append(pose)
    pose_info = np.array(pose_info)
    pose_info = pose_info.astype("float64")
    pose_info = [np.vstack((x, np.array([0, 0, 0, 1]))) for x in pose_info]
    pose_info = np.array(pose_info)
    return pose_info

In [4]:
## Get Disparity b/w left and right images
def get_disparity(image_l, image_r):
    window_size = 5
    min_disp = 16
    num_disp = 144
    stereo = cv2.StereoSGBM_create(
        minDisparity=min_disp,
        numDisparities=num_disp,
        blockSize=5,
        P1=8 * 3 * window_size ** 2,
        P2=32 * 3 * window_size ** 2,
        disp12MaxDiff=1,
        uniquenessRatio=32,
        speckleWindowSize=700,
        speckleRange=3,
    )

    disparity_map = stereo.compute(image_l, image_r).astype(np.float32) / 16.0
    return disparity_map

In [5]:
def get_calibration_info():
    K = np.array(
        [
            [7.070912e02, 0.000000e00, 6.018873e02],
            [0.000000e00, 7.070912e02, 1.831104e02],
            [0.000000e00, 0.000000e00, 1.000000e00],
        ]
    )
    baseline = 0.53790448812
    K_dash = np.linalg.pinv(K)
    return K, K_dash, baseline

In [6]:
def to_world_frame(pcd, P):
    R = np.array([[-1, 0, 0], [0, -1, 0], [0, 0, 1]])
    data = np.asarray(pcd.points)
    data = transform_pose(data, P)
    data = rotate(R, data)
    color = np.asarray(pcd.colors)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(data)
    pcd.colors = o3d.utility.Vector3dVector(color)
    return pcd

def rotate(R, points):
    return (R @ points.T).T

def transform_pose(points, M):
    points = np.hstack((points, np.ones((points.shape[0], 1))))
    points = (M @ points.T).T
    points = points[:, 0:3] / points[:, 3, np.newaxis]
    return points

In [7]:
## Generate PCD for disparity Maps
def generate_pcd(image, disparity, Kinv, baseline, P):

    H, W = image.shape[:2]
    points = []
    colors = []
    min_disparity = disparity.min()
    focal_length = K[0][0]
    for i in range(H):
        for j in range(W):
            if disparity[i, j] > min_disparity:
                depth = (baseline*focal_length) / disparity[i, j]
                cameraPosition = Kinv @ np.array([[depth * j], [depth * i], [depth]])
                points.append(cameraPosition.flatten())
                colors.append(image[i, j] / 255.0)
    
    pcd_points = np.array(points)
    pcd_colors = np.array(colors)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcd_points)
    pcd.colors = o3d.utility.Vector3dVector(pcd_colors)
    
    world_frame_pcd = to_world_frame(pcd, P)
    return world_frame_pcd

In [8]:
image_left = get_images(left_image_path)
image_right = get_images(right_image_path)

poses = get_poses(pose_file)

In [9]:
K, K_dash, baseline = get_calibration_info()
print("K : \n", K)
print("K': \n", K_dash)
print("Baseline : ", baseline)
print()

full_pcd = []

for index, (image_l, image_r) in enumerate(zip(image_left, image_right)):
    disparity = get_disparity(image_l, image_r)
    
    pcd = generate_pcd(image_l, disparity, K_dash, baseline, poses[index])
    print(f"Processed stereo pair {index + 1:2d}!")
    
    full_pcd.append(pcd)

o3d.visualization.draw(full_pcd)

K : 
 [[707.09   0.   601.89]
 [  0.   707.09 183.11]
 [  0.     0.     1.  ]]
K': 
 [[ 1.41e-03  3.62e-19 -8.51e-01]
 [-1.59e-19  1.41e-03 -2.59e-01]
 [-6.56e-19 -4.25e-19  1.00e+00]]
Baseline :  0.53790448812

Processed stereo pair  1!
Processed stereo pair  2!
Processed stereo pair  3!
Processed stereo pair  4!
Processed stereo pair  5!
Processed stereo pair  6!
Processed stereo pair  7!
Processed stereo pair  8!
Processed stereo pair  9!
Processed stereo pair 10!
Processed stereo pair 11!
Processed stereo pair 12!
Processed stereo pair 13!
Processed stereo pair 14!
Processed stereo pair 15!
Processed stereo pair 16!
Processed stereo pair 17!
Processed stereo pair 18!
Processed stereo pair 19!
Processed stereo pair 20!
Processed stereo pair 21!


## PART 2: Motion estimation using iterative PnP

Using the generated reconstruction from the previous part, synthesize a new image taken by a virtualmonocular camera fixed at any arbitrary position and orientation.  Your task in this part is to recoverthis pose using an iterative Perspective-from-n-Points (PnP) algorithm. 

#### Procedure: 

<ol>
    <li> Obtain a set of 2D-3D correspondences between the the image and the point cloud.  Since hereyou’re generating the image, this should be easy to obtain. </li>
    <li> For this set of correspondences compute the total reprojection error c= $\sum_{i} ‖x_i−P_{k}X_i‖^2 $    where $P_{k}= K[R_{k}|t_{k}]$, $X_{i}$ is the 3D point in the world frame, $x_{i}$ is its corresponding projection. </li>
    <li> Solve for the pose $T_{k}$ that minimizes this non-linear reprojection error using a Gauss-Newton (GN)scheme.  Recall that in GN we start with some initial estimated value $x_{o}$ and iteratively refine the estimate using $x_{1}$= $∆x+x_0$, where $∆x$ is obtained by solving the normal equations $J^{T}J∆x$= -$J^{T}e$, until convergence.The main steps in this scheme are computing the corresponding Jacobians and updating the estimates correctly.  For our problem,  use a 12×1 vector parameterization for $T_{k}$(the top 3×4submatrix).  Run the optimization for different choices of initialization and report your observations. </li>
</ol>

Make sure that you write about how you calculate the residual and jacobians. Do not just include the code. The pose that you have been given is the ground truth, so using that will obviously give good results for optimization, so try out something else as well.

$$
\begin{align}
    K [R|t] X &= \begin{bmatrix}
        f_x && 0 && c_x\\
        0 && f_y && c_y\\
        0 && 0 && 1\\
    \end{bmatrix}
    \begin{bmatrix}
        a_1 && a_2 && a_3 && a_4\\
        a_5 && a_6 && a_7 && a_8\\
        a_9 && a_{10} && a_{11} && a_{12}\\
    \end{bmatrix}
    \begin{bmatrix}
        X_i\\
        Y_i\\
        Z_i\\
        1\\
    \end{bmatrix}\\
    &=
    \begin{bmatrix}
        f_x(X_ia_1 + Y_ia_2 + Z_ia_3 + a_4)+c_x(X_ia_9 + Y_ia_{10} + Z_ia_{11} + a_{12})\\
        f_y(X_ia_5 + Y_ia_6 + Z_ia_7 + a_8)+c_y(X_ia_9 + Y_ia_{10} + Z_ia_{11} + a_{12})\\
        X_ia_9 + Y_ia_{10} + Z_ia_{11} + a_{12}\\
    \end{bmatrix} \\
    \begin{bmatrix}
        x_i\\
        y_i\\
    \end{bmatrix} &=
    \begin{bmatrix}
        \cfrac{f_x(X_ia_1 + Y_ia_2 + Z_ia_3 + a_4)+c_x(X_ia_9 + Y_ia_{10} + Z_ia_{11} + a_{12})}{X_ia_9 + Y_ia_{10} + Z_ia_{11} + a_{12}}\\
        \cfrac{f_y(X_ia_5 + Y_ia_6 + Z_ia_7 + a_8)+c_y(X_ia_9 + Y_ia_{10} + Z_ia_{11} + a_{12})}{X_ia_9 + Y_ia_{10} + Z_ia_{11} + a_{12}}\\
    \end{bmatrix} \\ \\
    \text{Let, } \\
    u &= f_x \cdot (X_ia_1 + Y_ia_2 + Z_ia_3 + a_4)+c_x \cdot  (X_ia_9 + Y_ia_{10} + Z_ia_{11} + a_{12}) \\
    v &= f_y \cdot (X_ia_5 + Y_ia_6 + Z_ia_7 + a_8)+c_y \cdot (X_ia_9 + Y_ia_{10} + Z_ia_{11} + a_{12}) \\
    w &= X_ia_9 + Y_ia_{10} + Z_ia_{11} + a_{12} \\ 
    \lambda_x &= \frac{c_{x}w - u}{w^2} \\
    \lambda_y &= \frac{c_{y}w - v}{w^2} \\ \\
    &\textbf{Jacobian and Residual are as follows, } \\
    J_i &= \begin{bmatrix}
    \cfrac{\partial x_i}{\partial a_1} & \cfrac{\partial x_i}{\partial a_2} & \cfrac{\partial x_i}{\partial a_3} & \cfrac{\partial x_i}{\partial a_4} & \cfrac{\partial x_i}{\partial a_5} & \cfrac{\partial x_i}{\partial a_6} & \cfrac{\partial x_i}{\partial a_7} & \cfrac{\partial x_i}{\partial a_8} & \cfrac{\partial x_i}{\partial a_9} & \cfrac{\partial x_i}{\partial a_{10}} & \cfrac{\partial x_i}{\partial a_{11}} & \cfrac{\partial x_i}{\partial a_{12}}\\
    \cfrac{\partial y_i}{\partial a_1} & \cfrac{\partial y_i}{\partial a_2} & \cfrac{\partial y_i}{\partial a_3} & \cfrac{\partial y_i}{\partial a_4} & \cfrac{\partial y_i}{\partial a_5} & \cfrac{\partial y_i}{\partial a_6} & \cfrac{\partial y_i}{\partial a_7} & \cfrac{\partial y_i}{\partial a_8} & \cfrac{\partial y_i}{\partial a_9} & \cfrac{\partial y_i}{\partial a_{10}} & \cfrac{\partial y_i}{\partial a_{11}} & \cfrac{\partial y_i}{\partial a_{12}}\\
\end{bmatrix}_{~2\times12} \\ \\
J_i &= \begin{bmatrix}
    \cfrac{f_x X_i}{w} & \cfrac{f_x Y_i}{w} & \cfrac{f_x Z_i}{w} & \cfrac{f_x}{w} & 0 & 0 & 0 & 0 & \lambda_x X_i & \lambda_x Y_i & \lambda_x Z_i & \lambda_x\\
    0 & 0 & 0 & 0 & \cfrac{f_y X_i}{w} & \cfrac{f_y Y_i}{w} & \cfrac{f_y Z_i}{w} & \cfrac{f_y}{w} & \lambda_y X_i & \lambda_y Y_i & \lambda_y Z_i & \lambda_y\\
\end{bmatrix}_{~2\times12} \\ \\
R_i &= \begin{bmatrix}
    x_i - \cfrac{u}{w} \\ 
    y_i - \cfrac{v}{w} \\ 
    \end{bmatrix}_{~2\times1}
\end{align}
$$


In [10]:
k_index = np.random.choice(len(full_pcd))

K, K_dash, baseline = get_calibration_info()

P_init = np.round(poses[k_index][:3, :], decimals=0) * np.random.normal(0, 0.5, size=(3, 4))

# P_init = np.random.rand(3, 4)

P_gt = poses[k_index][:3, :]

In [11]:
## 2D-3D correspondences
pcd = full_pcd[k_index]
pcd_points = np.array(pcd.points)

indices = np.random.choice(pcd_points.shape[0], size=(2000,), replace=False)
pcd_points = pcd_points[indices]

points3d = np.concatenate((pcd_points, np.ones((pcd_points.shape[0], 1))), axis=1)
points3d = points3d

points2d = K @ (P_gt @ points3d.T)
points2d = points2d / points2d[2, :]
points2d = points2d.T

In [12]:
def reprojection_error(x_i, x_proj):
    error = np.sum(np.linalg.norm(x_i - x_proj, axis=0) ** 2)
    return error

def compute_residual(x_i, X_i, P_k):
    x_homo = x_i / x_i[:, -1][:, None]
    X_homo = (P_k @ X_i.T).T
    X_homo = X_homo / X_homo[:, -1][:, None]
    res = x_homo[:, :-1] - X_homo[:, :-1]
    return res.reshape(-1, 1)

def compute_jacobian(K, P, X):
    J = np.zeros((2 * X.shape[0], 12))
    f_x, f_y, c_x, c_y = K[0, 0], K[1, 1], K[0, 2], K[1, 2]
    for i in range(0, X.shape[0]):
        u, v, w = K @ (P @ X[i])
        Xc = (c_x * w - u) / (w ** 2)
        Yc = (c_y * w - v) / (w ** 2)
        Fx = f_x / w
        Fy = f_y / w
        for k in range(0, 4):
            J[2 * i, k] = X[i, k] * Fx
            J[2 * i + 1, 4 + k] = X[i, k] * Fy
            J[2 * i, 8 + k] = X[i, k] * Xc
            J[2 * i + 1, 8 + k] = X[i, k] * Yc
    return np.array(J)

def project_points(P, X):
    x_proj = (P @ X.T).T
    x_proj = x_proj / x_proj[:, -1][:, None]
    return x_proj

In [13]:
x_proj = project_points(K @ P_init, points3d)
error = reprojection_error(points2d, x_proj)
print(f"Reprojection Error with P_init: {error:.7f}")

Reprojection Error with P_init: 17131318101.3851585


In [14]:
x_proj = project_points(K @ P_gt, points3d)
error = reprojection_error(points2d, x_proj)
print(f"Reprojection Error with P_gt: {error:.7f}")

Reprojection Error with P_gt: 0.0000000


In [15]:
def gauss_newton(P_init, P_gt, K, x, X, lr = 0.08, max_steps=1000):
    P = P_init.copy()

    x_proj = project_points(K @ P, X)

    error = reprojection_error(x, x_proj)
    step = 0
    while error > 1e-7 and step < max_steps:
        J_r = compute_jacobian(K, P, X)
        r = compute_residual(x, X, K @ P)

        J_F = np.linalg.pinv(J_r.T @ J_r) @ J_r.T @ r
        delta = -J_F
        delta = delta.reshape(3, 4)

        P -= lr * delta

        x_proj = project_points(K @ P, X)

        error = reprojection_error(x, x_proj)
        step += 1

        if step % 50 == 0:
            print(f"Step: {step:3d}, Error: {error:.7f}")

    if step < max_steps:
        print(f"Converged in {step} steps!!")
    else:
        print("Did not Converge!!")
    return P


P_pred = gauss_newton(P_init, P_gt, K, points2d, points3d)
print("===========================================")
print("Initial Pose:\n", P_init)
print("===========================================")
print("Ground Truth Pose:\n", P_gt)
print("===========================================")
print("Final Pose:\n", P_pred)
print("===========================================")

Step:  50, Error: 3823896.3199262
Step: 100, Error: 912.5725264
Step: 150, Error: 0.2182903
Step: 200, Error: 0.0000522
Converged in 238 steps!!
Initial Pose:
 [[ 0.11 -0.   -0.59 13.91]
 [-0.    0.6   0.   -0.37]
 [ 0.31  0.   -0.2  43.12]]
Ground Truth Pose:
 [[-8.61e-01  2.16e-02  5.08e-01 -1.86e+02]
 [ 4.41e-02  9.99e-01  3.21e-02  1.90e+00]
 [-5.07e-01  5.01e-02 -8.61e-01  4.52e+01]]
Final Pose:
 [[ 1.29e+00 -3.25e-02 -7.64e-01  2.79e+02]
 [-6.62e-02 -1.50e+00 -4.83e-02 -2.86e+00]
 [ 7.62e-01 -7.52e-02  1.29e+00 -6.80e+01]]


## PART 3: Odometry Calculation

In part 1, you used the ground truth pose for registration. Here, try to estimate the pose using the RGB image data alone. 

#### Procedure:

1. This can be done by computing features across the two images and matching them. Since you already have the depth map, you now have correspondences between the depth maps of two images as well from the RGB feature matches. 
2. You can now convert this depth map to a point cloud.
3. Since you have correspondences between image points in the depth map, you have 3D correspondences here as well. Perform ICP here to get a good pose estimate.
4. Feed these initial pose estimates into the PnP pipeline and optimise further.